# Summary

Previous notebook with EDA, modelling and manual hyper-parameter tuning:  
[Ventilator Pressure Prediction - LGB EDA](https://www.kaggle.com/squarex/ventilator-pressure-prediction-lgb-eda)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, impute
from IPython.display import display

from sklearn.dummy import DummyRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict, cross_validate
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.linear_model import Ridge, RidgeCV

from fbprophet import Prophet
import optuna
import statsmodels.api as sm
import scipy.stats as stats

plt.style.use('ggplot')

In [ ]:
random_state = 42
rng = np.random.RandomState(random_state)

In [ ]:
X = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv', index_col=0)
X_test = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv', index_col=0)

In [ ]:
print(X.columns.difference(X_test.columns))
y = X.pop('pressure')

In [ ]:
# num_columns = ['R', 'C', 'u_in', 'u_out']
ds = 'time_step'
ts_id = 'breath_id'

# Baseline

In [ ]:
def get_folds(X, y, n_splits=5, ts_id=ts_id):
    # return fixed list of CV index (for XGB, LGB compability to groups)
    folds = GroupKFold(n_splits=n_splits)
    return list(folds.split(X, y, groups=X[ts_id]))


def score_model(model, X=X, y=y, folds=None, fit_as_score=True):
    if folds is None:
        folds = get_folds(X, y)
        
    sample_weight = X.mask(lambda x: x['u_out'] == 0).isna().any(axis=1).astype(int)
    
    def scorer(y_true, y_pred, sample_weight=sample_weight):
        return mean_absolute_error(y_true.values, y_pred, sample_weight=sample_weight.loc[y_true.index].values)
    scoring = make_scorer(scorer, greater_is_better=False, sample_weight=sample_weight)
    
    if fit_as_score:
        fit_params = {'sample_weight':sample_weight}
    else:
        fit_params = {}
        
    d = cross_validate(model, X, y, scoring=scoring, cv=folds, n_jobs=1, fit_params=fit_params, return_train_score=True)
    d['train_score'] = -d['train_score']
    d['test_score'] = -d['test_score']
    return pd.DataFrame(d).style.bar(subset=['fit_time', 'score_time'], vmin=0, vmax=30)\
                                .bar(subset=['test_score', 'train_score'], vmin=0, vmax=7.5)

# Processing

In [ ]:
def add_lag(X, k=1, columns=['u_in', 'time_step'], fill=0):
    X = X.copy()
    if isinstance(k, int):
        k = list(range(1, k+1))
    if columns is None:
        columns = X.columns.tolist()
    
    for i in k:
        X_lag = X[columns].groupby(X[ts_id]).shift(i)
        if fill is not None:
            X_lag = X_lag.fillna(fill)
        X = X.join(X_lag.add_prefix(f'lag_{i}_'))
        
    return X


def proc(X):
    pi = np.pi
    X = X.eval('''RC=R*C/1000
                  tau=2 * @pi * RC
                  t=time_step/tau
                  et=1-exp(-t)
                  uet=u_in*et''')\
         .assign(time_diff=lambda x: x.groupby(ts_id)['time_step'].diff().fillna(0),
                 int_u_in=lambda x: x.eval('u_in*time_diff').groupby(x[ts_id]).cumsum(),
                 sum_u_in=lambda x: x.groupby(ts_id)['u_in'].cumsum(),
                 diff_u_in=lambda x: x.groupby(ts_id)['u_in'].diff().fillna(0),
                 max_u_in=lambda x: x.groupby(ts_id)['u_in'].transform('max'),)
    return X

# Compare to baseline

# Hyperparameter tuning

In [ ]:
X_proc = add_lag(proc(X).query('u_out==0'), k=[2], columns=['u_in', 'time_step', 'int_u_in', 'sum_u_in'])
y_proc = y.loc[X['u_out'] == 0]

# select categorical as 1 value at each "breath_id"
categorical_feature = [ts_id] + (X_proc.groupby(ts_id).nunique().mean() == 1).loc[lambda x: x].index.tolist()
# X_post[categorical_features] = pd.Categorical(X_post[categorical_features])

Best manually optimized learning rate

In [ ]:
params = {
    "objective": "regression_l1",
    "random_state": random_state,
    "metric": "l1",
    "verbosity": -1,
    #"learning_rate": 0.36, #leave it by default
    "num_threads":4,
}

Hyperparameter-tuning improve score less than just increasing the number of estimators to infinite.
(The model struggles to overfit so we can push n_estimators.)

In [ ]:
# from optuna.integration import lightgbm as lgb_opt

# dtrain = lgb_opt.Dataset(X_proc, y_proc)

# study = optuna.create_study()
# lgb_tuner = lgb_opt.LightGBMTunerCV(
#     params=params,
#     train_set=dtrain,
#     folds=get_folds(X_proc, y_proc),
#     shuffle=False,
#     stratified=False,
#     #return_cvbooster=True,
#     verbose_eval=-1,
#     show_progress_bar=True,
#     optuna_seed=random_state,
#     early_stopping_rounds=100,
#     num_boost_round=500,#1_000,
#     study=study,
# )
# lgb_tuner.run()

In [ ]:
# optuna.visualization.plot_optimization_history(study)

In [ ]:
# print("Best score:", lgb_tuner.best_score)
# best_params = lgb_tuner.best_params
# print("Best params:", best_params)
# print("  Params: ")
# for key, value in best_params.items():
#     print("    {}: {}".format(key, value))
    
# # print("best iteration: ", lgb_tuner.get_best_booster().best_iteration)

In [ ]:
# params = lgb_tuner.best_params
params['n_estimators'] = 100_000 # int(lgb_tuner.get_best_booster().best_iteration*1.1)
params.pop('num_threads')

print(params)

In [ ]:
# score_model(lgb.LGBMRegressor(**params), X=X_proc, y=y_proc)

Results with n_estimators = 10_000
```
 	fit_time	score_time	test_score	train_score
0	902.430400	190.816195	0.734996	0.497027
1	935.501294	116.737442	0.732414	0.501023
2	877.202830	106.261424	0.739981	0.495680
3	899.060446	196.055113	0.735223	0.500615
4	897.823282	133.426292	0.736875	0.495777
```

# Submission

In [ ]:
X_test_proc = add_lag(proc(X_test).query('u_out==0'), k=[2], columns=['u_in', 'time_step', 'int_u_in', 'sum_u_in'])

In [ ]:
model = lgb.LGBMRegressor(**params).fit(X_proc, y_proc)
y_pred = model.predict(X_test_proc)

In [ ]:
y_s = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/sample_submission.csv', index_col=0)
y_s.loc[X_test_proc.index] = y_pred

y_s.to_csv("submission.csv")